# Regularization Effects on Parallel Circuits

This notebook implements experiments to test how L1, L2, and dropout regularization affect parallel circuit emergence in neural networks.

**Hypotheses:**
- **L1 regularization** → Sparsity → Fewer parallel circuits
- **L2 regularization** → Weight diffusion → More parallel circuits  
- **Dropout** → Forced redundancy → More parallel circuits

---

## Setup

First, let's verify GPU availability and clone the repository.

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    device = 'cuda:0'
else:
    print("Warning: CUDA not available, using CPU")
    device = 'cpu'

In [ ]:
# Clone the MI-identifiability repository
!git clone https://github.com/MelouxM/MI-identifiability.git
%cd MI-identifiability

# Install dependencies
!pip install -q -e .

## Code Modifications

Now we'll modify the code to add regularization support.

In [ ]:
# Backup original files
!cp mi_identifiability/neural_model.py mi_identifiability/neural_model.py.bak
!cp main.py main.py.bak
print("Original files backed up!")

In [ ]:
%%writefile mi_identifiability/neural_model.py
import copy
import itertools
from collections import defaultdict
from typing import Dict, Any

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset

ACTIVATION_FUNCTIONS = {
    'leaky_relu': nn.LeakyReLU,
    'relu': nn.ReLU,
    'tanh': nn.Tanh,
    'sigmoid': nn.Sigmoid
}


class MLP(nn.Module):
    """
    A class implementing a simple multi-layer perceptron model.
    """
    def __init__(self, hidden_sizes: list, input_size=2, output_size=1, 
                 activation='leaky_relu', device='cpu', dropout_rate=0.0):
        """
        Initialize the MLP model.

        Args:
            hidden_sizes: A list of sizes of the hidden layers (in neurons), not including the input and output layers
            input_size: The size of the input layer
            output_size: The size of the output layer
            activation: The activation function to use
            device: The device to run the model on
            dropout_rate: Dropout probability (0.0 = no dropout)
        """
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.layer_sizes = [input_size] + hidden_sizes + [output_size]
        self.activation = activation
        self.dropout_rate = dropout_rate

        # Build layers with optional dropout
        self.layers = nn.ModuleList()
        for idx, (in_size, out_size) in enumerate(zip(self.layer_sizes, self.layer_sizes[1:])):
            layer_components = [nn.Linear(in_size, out_size)]
            
            # Add dropout after linear layer (but not on output layer)
            if idx < len(self.layer_sizes) - 2 and dropout_rate > 0:
                layer_components.append(nn.Dropout(dropout_rate))
            
            # Add activation function (Identity for output layer)
            if idx < len(self.layer_sizes) - 2:
                layer_components.append(ACTIVATION_FUNCTIONS[activation]())
            else:
                layer_components.append(nn.Identity())
            
            self.layers.append(nn.Sequential(*layer_components))

        self.num_layers = len(self.layers)
        self.device = device
        self.to(device)

    def save_to_file(self, filepath):
        """
        Save the model's configuration and state_dict to a file.
        """
        torch.save({
            'input_size': self.input_size,
            'hidden_sizes': self.hidden_sizes,
            'output_size': self.output_size,
            'activation': self.activation,
            'dropout_rate': self.dropout_rate,
            'state_dict': self.state_dict()
        }, filepath)

    @classmethod
    def load_from_file(cls, filepath):
        """
        Load a model from a file and return it.
        """
        model_data = torch.load(filepath)
        model = cls(
            model_data['hidden_sizes'],
            input_size=model_data['input_size'],
            output_size=model_data['output_size'],
            activation=model_data['activation'],
            dropout_rate=model_data.get('dropout_rate', 0.0)
        )
        model.load_state_dict(model_data['state_dict'])
        return model

    def out_features(self, layer_id):
        """
        Helper function to get the output size of a given layer

        Args:
            layer_id: The index of the layer

        Returns:
            The output size of the layer
        """
        return self.layers[layer_id][0].out_features

    def forward(self, x, circuit=None, interventions=None, return_activations=False):
        """
        Applies a forward pass through the model, optionally using only a subset of the model (circuit) and applying
        interventions.

        Args:
            x: The input tensor
            circuit: The circuit to apply
            interventions: The list of interventions to apply
            return_activations: Whether to return the activations of each layer

        Returns:
            The output of the model if return_activations is False, else the activations of each layer
        """
        activations = [x.detach()] if return_activations else None

        # Prepare the list of interventions, ordering it by layer
        ordered_interventions = defaultdict(list)
        if interventions is not None:
            for (layer_idx, indices), values in interventions.items():
                ordered_interventions[layer_idx].append((indices, values))

        if 0 in ordered_interventions:
            for indices, values in ordered_interventions[0]:
                x[list(indices)] = values

        original_weights = []  # Store original weights if a circuit is applied

        for i, layer in enumerate(self.layers):
            if circuit:
                # Store original weights before applying circuit edge masks
                original_weights.append(layer[0].weight.data.clone())
                layer[0].weight.data *= torch.tensor(
                    circuit.edge_masks[i], dtype=torch.float32, device=self.device
                )
                if circuit.node_masks:
                    x *= torch.tensor(
                        circuit.node_masks[i], dtype=torch.float32, device=self.device
                    )

            # Forward pass
            x = layer(x)

            if return_activations:
                activations.append(x.detach())

            # Apply interventions at the current layer
            if interventions and i + 1 in ordered_interventions:
                for indices, values in ordered_interventions[i + 1]:
                    x[list(indices)] = values

        if circuit:
            # Restore original weights
            for i, layer in enumerate(self.layers):
                layer[0].weight.data = original_weights[i]

        return activations if return_activations else x

    def get_states(self, x, states: Dict[Any, Any]) -> Dict[Any, torch.Tensor]:
        """
        Runs an input through the model and returns selected hidden states.

        Args:
            x: The input tensor
            states: A dictionary of tuples (layer, indices) to return

        Returns:
            A dictionary mapping each tuple (layer, indices) to the corresponding hidden states
        """
        activations = self(x, return_activations=True)
        return {
            (layer, tuple(indices)): activations[layer][indices]
            for layer, indices in states.items()
        }

    def visualize(self, ax=None, circuit=None, activations=None):
        """
        Visualize the model's structure or activations

        Args:
            ax: The axis to plot on
            circuit: The optional circuit to visualize
            activations: The optional activations to visualize
        """

        G = nx.DiGraph()
        pos = {}
        colors = {}
        node_labels = {}
        edge_labels = {}
        use_activation_values = True

        # Set default values
        if circuit is None:
            from .circuit import Circuit

            circuit = Circuit.full(self.layer_sizes)
        node_masks = circuit.node_masks
        edge_masks = circuit.edge_masks

        if activations is None:
            activations = [torch.ones(1, size) for size in self.layer_sizes]
            use_activation_values = False

        # Build the graph
        max_width = max(self.layer_sizes)

        for layer_idx, layer_activations in enumerate(activations):
            num_nodes = layer_activations.shape[-1]
            y_start = -(max_width - num_nodes) / 2  # Center nodes vertically

            for node_idx in range(num_nodes):
                node_id = f"({layer_idx},{node_idx})"
                G.add_node(node_id)
                pos[node_id] = (layer_idx, y_start - node_idx)

                # Apply node mask for visualization
                if node_masks is not None:
                    active = node_masks[layer_idx][node_idx].item() == 1
                else:
                    active = True

                if use_activation_values:
                    activation_value = layer_activations[0, node_idx].item()
                    node_labels[node_id] = f"{activation_value:.2f}"
                    color_intensity = np.clip(activation_value, 0, 1)  # Normalize to [0, 1]
                    node_color = plt.cm.YlOrRd(color_intensity) if active else 'grey'  # Use heatmap for activations
                else:
                    node_color = 'tab:blue' if active else 'grey'
                colors[node_id] = node_color

        # Add edges to the graph and create edge labels
        for layer_idx, edge_mask in enumerate(edge_masks):
            for out_idx, row in enumerate(edge_mask):
                for in_idx, active in enumerate(row):
                    from_node_id = f"({layer_idx},{in_idx})"
                    to_node_id = f"({layer_idx + 1},{out_idx})"
                    G.add_edge(from_node_id, to_node_id, active=active.item())

                    weight = self.layers[layer_idx][0].weight[out_idx, in_idx].item()
                    edge_labels[(from_node_id, to_node_id)] = f"{weight:.2f}"

        # Draw nodes with color corresponding to activation value
        node_colors = [colors[node] for node in G.nodes]
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=1400, alpha=0.8)

        # Draw edges
        active_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr['active'] == 1]
        inactive_edges = [(u, v) for u, v, attr in G.edges(data=True) if attr['active'] == 0]
        nx.draw_networkx_edges(G, pos, node_size=1400, edgelist=active_edges,
                               edge_color='tab:red' if use_activation_values else 'tab:blue', width=1, alpha=0.6,
                               arrows=True, arrowstyle="-|>", connectionstyle='arc3,rad=0.1', ax=ax)
        nx.draw_networkx_edges(G, pos, node_size=1400, edgelist=inactive_edges, edge_color='grey', width=1, alpha=0.5,
                               style='dashed', arrows=True, arrowstyle="-|>", connectionstyle='arc3,rad=0.1', ax=ax)

        # Draw node labels (activation values)
        nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=15, font_color='black', alpha=0.8, ax=ax)

        # Draw edge labels (weights)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=7, alpha=0.7, label_pos=0.6, ax=ax)

        if use_activation_values:
            plt.title("Neural Network Execution Visualization")
        else:
            plt.title("Neural Network Visualization")
        plt.axis('off')
        plt.show()

    def __getitem__(self, idx, in_place=False):
        """
        Overload the indexing operator to create a submodel

        Args:
            idx: The index or slice to use for the submodel
            in_place: Whether to modify the original model weights in-place

        Returns:
            A new submodel, with shared weights if in_place is True, or a copy otherwise
        """
        if isinstance(idx, slice):
            # Handle negative indices in the slice
            start_idx = idx.start if idx.start is not None else 0
            stop_idx = idx.stop if idx.stop is not None else len(self.layers)
            step_idx = idx.step if idx.step is not None else 1

            # Adjust negative indices
            if start_idx < 0:
                start_idx += len(self.layers)
            if stop_idx < 0:
                stop_idx += len(self.layers)

            # Extract the sliced layers
            sliced_layers = self.layers[start_idx:stop_idx:step_idx]
            num_sliced_layers = len(sliced_layers)

            # Calculate the input size for the submodel (first layer in the slice)
            input_size = self.layer_sizes[start_idx]

            # Calculate the hidden sizes and output size for the submodel
            smaller_hidden_sizes = [layer[0].out_features for layer in sliced_layers[:-1]]
            output_size = sliced_layers[-1][0].out_features

            # Create a new submodel with the correct architecture
            submodel = MLP(
                hidden_sizes=smaller_hidden_sizes,
                input_size=input_size,
                output_size=output_size,
                activation=self.activation,
                device=self.device,
            )

            # Copy weights and biases from the original model
            for i, layer in enumerate(sliced_layers):
                layer_data = submodel.layers[i][0]
                layer_data.weight.data = layer[0].weight.data
                layer_data.bias.data = layer[0].bias.data
                if not in_place:
                    layer_data.weight.data = layer_data.weight.data.clone()
                    layer_data.bias.data = layer_data.bias.data.clone()

            return submodel
        else:
            # Handle single index
            if in_place:
                return self.layers[idx]
            return copy.deepcopy(self.layers[idx])

    def do_train(self, x, y, x_val, y_val, batch_size, learning_rate, epochs, loss_target=0.001, val_frequency=10,
                 early_stopping_steps=3, logger=None, l1_lambda=0.0, l2_lambda=0.0):
        """
        Train the model using the given data and hyperparameters.

        Args:
            x: The training input tensor
            y: The training target tensor
            x_val: The validation input tensor
            y_val: The validation target tensor
            batch_size: The batch size for training
            learning_rate: The learning rate for training
            epochs: The number of epochs to train
            loss_target: The target loss value to stop training
            val_frequency: The frequency of validation during training
            early_stopping_steps: The number of epochs without improvement to stop training
            logger: The logger to log training progress
            l1_lambda: L1 regularization coefficient (0.0 = no L1)
            l2_lambda: L2 regularization coefficient (0.0 = no L2)

        Returns:
            The average loss after training
        """
        # Create a DataLoader for the training dataset
        dataset = TensorDataset(x, y)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        # Define loss function and optimizer
        criterion = nn.MSELoss()
        optimizer = optim.AdamW(self.parameters(), lr=learning_rate)

        best_loss = float('inf')
        bad_epochs = 0

        val_acc = 0

        # Training loop
        for epoch in range(epochs):
            self.train()
            epoch_loss = []
            for inputs, targets in dataloader:
                optimizer.zero_grad()
                outputs = self(inputs)
                
                # Base loss
                base_loss = criterion(outputs, targets)
                loss = base_loss
                
                # Add L1 Regularization
                if l1_lambda > 0:
                    l1_penalty = sum(torch.sum(torch.abs(param)) for param in self.parameters())
                    loss = loss + l1_lambda * l1_penalty
                
                # Add L2 Regularization
                if l2_lambda > 0:
                    l2_penalty = sum(torch.sum(param ** 2) for param in self.parameters())
                    loss = loss + l2_lambda * l2_penalty
                
                loss.backward()
                optimizer.step()
                epoch_loss.append(loss.item())

            avg_loss = np.mean(epoch_loss)

            if avg_loss < best_loss:
                best_loss = avg_loss
                bad_epochs = 0
            else:
                bad_epochs += 1

            # Early stopping
            if avg_loss < loss_target or bad_epochs >= early_stopping_steps:
                break

            # Print training progress
            if (epoch + 1) % val_frequency == 0 and logger is not None:
                self.eval()
                with torch.no_grad():
                    train_outputs = self(x)
                    train_predictions = torch.round(train_outputs)
                    correct_predictions_train = train_predictions.eq(y).all(dim=1)
                    train_acc = correct_predictions_train.sum().item() / y.size(0)

                    val_outputs = self(x_val)
                    val_loss = criterion(val_outputs, y_val).item()
                    val_predictions = torch.round(val_outputs)
                    correct_predictions_val = val_predictions.eq(y_val).all(dim=1)
                    val_acc = correct_predictions_val.sum().item() / y_val.size(0)

                    # Log regularization info
                    reg_info = ""
                    if l1_lambda > 0:
                        reg_info += f", L1: {l1_lambda}"
                    if l2_lambda > 0:
                        reg_info += f", L2: {l2_lambda}"
                    if self.dropout_rate > 0:
                        reg_info += f", Dropout: {self.dropout_rate}"

                    logger.info(f'Epoch [{epoch + 1}/{epochs}], '
                                f'Train Loss: {avg_loss:.4f}, Train Accuracy: {train_acc:.4f}')
                    logger.info(f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}, Bad Epochs: {bad_epochs}{reg_info}')

        return avg_loss

    def do_eval(self, x_test, y_test):
        """
        Performs evaluation on the given test data

        Args:
            x_test: The test input tensor
            y_test: The test target tensor

        Returns:
            The accuracy of the model on the test data
        """
        self.eval()
        with torch.no_grad():
            val_outputs = self(x_test)
            val_predictions = torch.round(val_outputs)
            correct_predictions_val = val_predictions.eq(y_test).all(dim=1)
            acc = correct_predictions_val.sum().item() / y_test.size(0)
        return acc

    def separate_into_k_mlps(self):
        """
        Separates the original MLP into K individual MLPs, each with output size 1.

        Returns:
            A list of K MLP models, each for a single output.
        """
        separate_models = [self[:] for _ in range(self.output_size)]

        last_layer = self.layers[-1]
        last_layer_weights = last_layer[0].weight.data
        last_layer_biases = last_layer[0].bias.data

        for i, model in enumerate(separate_models):
            model.layers[-1][0] = nn.Linear(self.hidden_sizes[-1], 1)
            model.layers[-1][0].weight.data = last_layer_weights[i:i + 1, :].clone()
            model.layers[-1][0].bias.data = last_layer_biases[i:i + 1].clone()
            model.output_size = 1
            model.layer_sizes = model.layer_sizes[:-1] + [1]

        return separate_models

    def enumerate_valid_node_masks(self):
        """
        Generate all valid node masks in the neural network.

        Returns:
            A list of all valid node masks
        """
        # Generate all valid masks for each layer
        all_masks_per_layer = []

        for size in self.layer_sizes[1:]:
            masks = [np.array([int(x) for x in format(i, f'0{size}b')]) for i in range(2 ** size)]
            all_masks_per_layer.append(masks)

        # Generate combinations of masks for all layers
        all_masks = list(itertools.product(*all_masks_per_layer))
        return all_masks

Now let's modify the main.py file to accept regularization parameters:

In [ ]:
# Read the original main.py
with open('main.py', 'r') as f:
    main_content = f.read()

# Add regularization arguments to argparse section
import_section = main_content.split('if __name__ == "__main__":')[0]
main_section = main_content.split('if __name__ == "__main__":')[1]

# Find where to insert new arguments (after --verbose)
insert_pos = main_section.find("parser.add_argument('--resume-from'")

new_args = '''
    parser.add_argument('--l1-lambda', type=float, default=0.0,
                        help='L1 regularization coefficient (default: 0.0)')
    parser.add_argument('--l2-lambda', type=float, default=0.0,
                        help='L2 regularization coefficient (default: 0.0)')
    parser.add_argument('--dropout-rate', type=float, default=0.0,
                        help='Dropout rate, 0.0 to 1.0 (default: 0.0)')
    '''

main_section = main_section[:insert_pos] + new_args + main_section[insert_pos:]

# Modify model creation to include dropout
main_section = main_section.replace(
    'model = MLP(hidden_sizes=layer_sizes[1:-1], input_size=n_inputs, output_size=n_gates, device=args.device)',
    'model = MLP(hidden_sizes=layer_sizes[1:-1], input_size=n_inputs, output_size=n_gates, device=args.device, dropout_rate=args.dropout_rate)'
)

# Modify training call to include regularization
main_section = main_section.replace(
    '''avg_loss = model.do_train(
                x=x,
                y=y,
                x_val=x_val,
                y_val=y_val,
                batch_size=args.batch_size,
                learning_rate=lr,
                epochs=args.epochs,
                loss_target=loss_target,
                val_frequency=args.val_frequency,
                logger=logger if args.verbose else None
            )''',
    '''avg_loss = model.do_train(
                x=x,
                y=y,
                x_val=x_val,
                y_val=y_val,
                batch_size=args.batch_size,
                learning_rate=lr,
                epochs=args.epochs,
                loss_target=loss_target,
                val_frequency=args.val_frequency,
                logger=logger if args.verbose else None,
                l1_lambda=args.l1_lambda,
                l2_lambda=args.l2_lambda
            )'''
)

# Add regularization info to saved data
main_section = main_section.replace(
    "'weights': weights,",
    ''''weights': weights,
                'l1_lambda': args.l1_lambda,
                'l2_lambda': args.l2_lambda,
                'dropout_rate': args.dropout_rate'''
)

# Write modified main.py
with open('main.py', 'w') as f:
    f.write(import_section + 'if __name__ == "__main__":' + main_section)

print("✓ main.py modified successfully!")

## Run Experiments

Now we're ready to run experiments! Let's start with a quick test to make sure everything works.

In [ ]:
# Quick test run with baseline (no regularization)
!python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
  --n-experiments 3 --size 3 --depth 2 --device {device}

print("\n✓ Test run completed! Now you can run full experiments.")

### Baseline Experiment (No Regularization)

In [ ]:
# Baseline - 30 experiments (reduced from 100 for Colab)
!python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
  --n-experiments 30 --size 3 --depth 2 --device {device}

print("\n✓ Baseline experiment completed!")

### L1 Regularization Experiments

In [ ]:
# L1 Experiments
l1_lambdas = [0.0001, 0.001, 0.01]

for l1 in l1_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L1 experiment with lambda={l1}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 30 --size 3 --depth 2 --device {device} --l1-lambda {l1}

print("\n✓ All L1 experiments completed!")

### L2 Regularization Experiments

In [ ]:
# L2 Experiments
l2_lambdas = [0.0001, 0.001, 0.01]

for l2 in l2_lambdas:
    print(f"\n{'='*60}")
    print(f"Running L2 experiment with lambda={l2}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 30 --size 3 --depth 2 --device {device} --l2-lambda {l2}

print("\n✓ All L2 experiments completed!")

### Dropout Experiments

In [ ]:
# Dropout Experiments
dropout_rates = [0.1, 0.3, 0.5]

for dr in dropout_rates:
    print(f"\n{'='*60}")
    print(f"Running Dropout experiment with rate={dr}")
    print(f"{'='*60}\n")
    
    !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
      --n-experiments 30 --size 3 --depth 2 --device {device} --dropout-rate {dr}

print("\n✓ All Dropout experiments completed!")

## Analysis

Now let's analyze the results!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Load all results
results_dir = Path('logs')
all_data = []

for csv_file in results_dir.glob("**/df_out.csv"):
    df = pd.read_csv(csv_file)
    all_data.append(df)

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"Loaded {len(combined_df)} experiment results")
    print(f"\nColumns: {combined_df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(combined_df.head())
else:
    print("No results found!")

In [ ]:
# Preprocess data - extract circuit counts from lists
import ast

def extract_first(x):
    if isinstance(x, str):
        try:
            x = ast.literal_eval(x)
        except:
            return x
    if isinstance(x, list) and len(x) > 0:
        return x[0]
    return x

combined_df['n_circuits'] = combined_df['perfect_circuits'].apply(extract_first)
combined_df['n_formulas'] = combined_df['formulas'].apply(extract_first)

# Ensure regularization columns exist with defaults
if 'l1_lambda' not in combined_df.columns:
    combined_df['l1_lambda'] = 0.0
if 'l2_lambda' not in combined_df.columns:
    combined_df['l2_lambda'] = 0.0
if 'dropout_rate' not in combined_df.columns:
    combined_df['dropout_rate'] = 0.0

print("Data preprocessed!")
print(f"\nCircuit count statistics:")
print(combined_df['n_circuits'].describe())

In [ ]:
# Plot L1 effects
if combined_df['l1_lambda'].nunique() > 1:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Box plot
    sns.boxplot(data=combined_df, x='l1_lambda', y='n_circuits', ax=axes[0])
    axes[0].set_title('L1 Regularization: Effect on Circuit Count', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('L1 Lambda', fontsize=12)
    axes[0].set_ylabel('Number of Circuits', fontsize=12)
    
    # Mean trend
    mean_circuits = combined_df.groupby('l1_lambda')['n_circuits'].mean()
    axes[1].plot(mean_circuits.index, mean_circuits.values, marker='o', linewidth=2, markersize=8)
    axes[1].set_title('L1 Regularization: Mean Circuit Count', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('L1 Lambda', fontsize=12)
    axes[1].set_ylabel('Mean Circuit Count', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('l1_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistical test
    baseline = combined_df[combined_df['l1_lambda'] == 0.0]['n_circuits']
    for l1 in combined_df['l1_lambda'].unique():
        if l1 > 0:
            treatment = combined_df[combined_df['l1_lambda'] == l1]['n_circuits']
            t_stat, p_val = stats.ttest_ind(baseline, treatment)
            print(f"L1={l1}: Mean change = {treatment.mean() - baseline.mean():.2f}, p={p_val:.4f}")
else:
    print("L1 experiments not found in data")

In [ ]:
# Plot L2 effects
if combined_df['l2_lambda'].nunique() > 1:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Box plot
    sns.boxplot(data=combined_df, x='l2_lambda', y='n_circuits', ax=axes[0])
    axes[0].set_title('L2 Regularization: Effect on Circuit Count', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('L2 Lambda', fontsize=12)
    axes[0].set_ylabel('Number of Circuits', fontsize=12)
    
    # Mean trend
    mean_circuits = combined_df.groupby('l2_lambda')['n_circuits'].mean()
    axes[1].plot(mean_circuits.index, mean_circuits.values, marker='o', linewidth=2, markersize=8, color='orange')
    axes[1].set_title('L2 Regularization: Mean Circuit Count', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('L2 Lambda', fontsize=12)
    axes[1].set_ylabel('Mean Circuit Count', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('l2_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistical test
    baseline = combined_df[combined_df['l2_lambda'] == 0.0]['n_circuits']
    for l2 in combined_df['l2_lambda'].unique():
        if l2 > 0:
            treatment = combined_df[combined_df['l2_lambda'] == l2]['n_circuits']
            t_stat, p_val = stats.ttest_ind(baseline, treatment)
            print(f"L2={l2}: Mean change = {treatment.mean() - baseline.mean():.2f}, p={p_val:.4f}")
else:
    print("L2 experiments not found in data")

In [ ]:
# Plot Dropout effects
if combined_df['dropout_rate'].nunique() > 1:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Box plot
    sns.boxplot(data=combined_df, x='dropout_rate', y='n_circuits', ax=axes[0])
    axes[0].set_title('Dropout: Effect on Circuit Count', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Dropout Rate', fontsize=12)
    axes[0].set_ylabel('Number of Circuits', fontsize=12)
    
    # Mean trend
    mean_circuits = combined_df.groupby('dropout_rate')['n_circuits'].mean()
    axes[1].plot(mean_circuits.index, mean_circuits.values, marker='o', linewidth=2, markersize=8, color='green')
    axes[1].set_title('Dropout: Mean Circuit Count', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Dropout Rate', fontsize=12)
    axes[1].set_ylabel('Mean Circuit Count', fontsize=12)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('dropout_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # Statistical test
    baseline = combined_df[combined_df['dropout_rate'] == 0.0]['n_circuits']
    for dr in combined_df['dropout_rate'].unique():
        if dr > 0:
            treatment = combined_df[combined_df['dropout_rate'] == dr]['n_circuits']
            t_stat, p_val = stats.ttest_ind(baseline, treatment)
            print(f"Dropout={dr}: Mean change = {treatment.mean() - baseline.mean():.2f}, p={p_val:.4f}")
else:
    print("Dropout experiments not found in data")

In [ ]:
# Summary statistics
print("\n" + "="*60)
print("SUMMARY: REGULARIZATION EFFECTS ON PARALLEL CIRCUITS")
print("="*60 + "\n")

baseline_circuits = combined_df[
    (combined_df['l1_lambda'] == 0) & 
    (combined_df['l2_lambda'] == 0) & 
    (combined_df['dropout_rate'] == 0)
]['n_circuits']

if len(baseline_circuits) > 0:
    print(f"Baseline (no regularization):")
    print(f"  Mean circuits: {baseline_circuits.mean():.2f} ± {baseline_circuits.std():.2f}")
    print(f"  Median: {baseline_circuits.median():.0f}")
    print(f"  Range: [{baseline_circuits.min():.0f}, {baseline_circuits.max():.0f}]")
    print()

# Check hypotheses
print("HYPOTHESIS TESTING:\n")

if combined_df['l1_lambda'].nunique() > 1:
    l1_max = combined_df[combined_df['l1_lambda'] > 0]['l1_lambda'].max()
    l1_circuits = combined_df[combined_df['l1_lambda'] == l1_max]['n_circuits']
    l1_change = l1_circuits.mean() - baseline_circuits.mean()
    print(f"L1 Hypothesis (sparsity → fewer circuits):")
    print(f"  Strongest L1={l1_max}: {l1_change:+.2f} circuits")
    print(f"  Direction: {'✓ SUPPORTED' if l1_change < 0 else '✗ NOT SUPPORTED'}")
    print()

if combined_df['l2_lambda'].nunique() > 1:
    l2_max = combined_df[combined_df['l2_lambda'] > 0]['l2_lambda'].max()
    l2_circuits = combined_df[combined_df['l2_lambda'] == l2_max]['n_circuits']
    l2_change = l2_circuits.mean() - baseline_circuits.mean()
    print(f"L2 Hypothesis (diffusion → more circuits):")
    print(f"  Strongest L2={l2_max}: {l2_change:+.2f} circuits")
    print(f"  Direction: {'✓ SUPPORTED' if l2_change > 0 else '✗ NOT SUPPORTED'}")
    print()

if combined_df['dropout_rate'].nunique() > 1:
    dr_max = combined_df[combined_df['dropout_rate'] > 0]['dropout_rate'].max()
    dr_circuits = combined_df[combined_df['dropout_rate'] == dr_max]['n_circuits']
    dr_change = dr_circuits.mean() - baseline_circuits.mean()
    print(f"Dropout Hypothesis (redundancy → more circuits):")
    print(f"  Strongest dropout={dr_max}: {dr_change:+.2f} circuits")
    print(f"  Direction: {'✓ SUPPORTED' if dr_change > 0 else '✗ NOT SUPPORTED'}")
    print()

print("="*60)

## Export Results

Let's download the results and plots to your local machine.

In [ ]:
# Save combined results
combined_df.to_csv('all_results.csv', index=False)
print("Results saved to all_results.csv")

# Create a zip file with all results
!zip -r regularization_results.zip logs/ *.png all_results.csv

print("\nAll results packaged in regularization_results.zip")
print("Download this file to your local machine!")

# In Colab, you can download with:
from google.colab import files
files.download('regularization_results.zip')

## Next Steps

### If you want to run more experiments:

1. **Test combined regularization:**
   ```python
   !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
     --n-experiments 30 --size 3 --depth 2 --device {device} \
     --l1-lambda 0.001 --l2-lambda 0.001 --dropout-rate 0.3
   ```

2. **Test on different tasks:**
   ```python
   !python main.py --verbose --val-frequency 1 --noise-std 0.0 \
     --target-logic-gates AND OR IMP \
     --n-experiments 30 --size 3 --depth 2 --device {device} --l2-lambda 0.001
   ```

3. **Test on larger networks:**
   ```python
   !python main.py --verbose --val-frequency 1 --noise-std 0.0 --target-logic-gates XOR \
     --n-experiments 30 --size 5 --depth 3 --device {device} --dropout-rate 0.3
   ```

### To continue analysis:
- Re-run the analysis cells after each new experiment
- The plots will automatically update with new data
- Download updated results using the export cell

### Remember:
- Colab sessions timeout after 12 hours of inactivity
- Download your results regularly!
- GPU availability may be limited on free tier